In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt
from scipy import stats,optimize
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import r2_score,mean_squared_error
import math


In [ ]:
#right now this gets data locally - will add API support soon

In [4]:
weather_spiro = pd.read_csv('weather_spiro.csv',parse_dates=['time'])
weather_spiro.head()

,barometricPressure,precipitationLast3Hours,relativeHumidity,temperature,time,Time,PEF(l/min),FEV1(l),unixDate,unixTime
0,103010.0,0.0,41.008829,0.0,2020-02-21 00:51:00,1900-01-01 00:08:00,415,2.89,18313.0,480.0
1,103150.0,0.0,40.326484,-2.2,2020-02-21 03:51:00,1900-01-01 03:31:00,388,2.54,18313.0,12660.0
2,102200.0,0.0,18.705038,10.0,2020-02-22 19:51:00,1900-01-01 19:32:00,476,2.78,18314.0,70320.0
3,102300.0,0.0,66.125542,0.6,2020-02-23 05:51:00,1900-01-01 05:13:00,294,1.62,18315.0,18780.0
4,102340.0,0.0,35.825672,9.4,2020-02-23 15:51:00,1900-01-01 15:37:00,384,2.49,18315.0,56220.0


In [5]:
#find what sorts of reports are available for what area
#https://docs.airnowapi.org/Data/query
#http://www.airnowapi.org/aq/data/?startDate=2020-02-21T13&endDate=2020-03-26T14&parameters=OZONE,PM25,PM10,CO,NO2,SO2&BBOX=-73.870000,40.696253,-73.792409,40.762596&dataType=B&format=text/csv&verbose=1&nowcastonly=0&includerawconcentrations=1&API_KEY=
#to-do learn api format https://docs.airnowapi.org/Data/docs
#get other pollution concentrations
colnames=["Latitude","Longitude","UTC","Parameter","Concentration","Unit","Raw Concentration","AQI","Category",
    "Site Name","Site Agency","AQS ID","Full AQS ID"] 
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%dT%H:%M')

#df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse)

AQI = pd.read_csv('./Output.csv',parse_dates=['UTC'], date_parser=dateparse,names=colnames, header=None)
AQI = AQI[["UTC","Parameter","Concentration","AQI","Site Name"]]
#we have hourly results but we won't use all that data yet
AQI['UTC']=AQI['UTC'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))
AQI['unixDate']= AQI['UTC'].astype(np.int64)// 10**9
AQI['unixDate'] = AQI['unixDate']/86400

AQITotal = AQI[['unixDate','AQI']]
AQITotal.index = AQITotal['unixDate']
AQITotal = AQITotal.groupby(AQITotal['unixDate']).mean()
#AQITotal = AQITotal.drop(['unixDate'], axis=1)
AQI = AQI.drop(['AQI'], axis=1)


In [7]:
def formatPollutionTables(pollutant,site=""):
    table = AQI.copy()
    table = table.where(table['Parameter']==pollutant)
    table = table.rename(columns={"Concentration":pollutant+site})
    if (pollutant == 'PM2.5'):
        table = table.where(table["Site Name"]==site).dropna()
    return table.groupby(table['unixDate']).mean()

ozone = formatPollutionTables("OZONE")
PM25 = formatPollutionTables("PM2.5","Queens")
PM25Road = formatPollutionTables("PM2.5","Queens Near-road")

In [9]:
PM25.head()

,PM2.5Queens
unixDate,
18313.0,3.372727
18314.0,7.404167
18315.0,10.737500
18316.0,13.283333
18317.0,9.429167


In [12]:
weather_spiro.index=weather_spiro['unixDate']

all_data = ozone.join(PM25).join(PM25Road).join(AQITotal).join(weather_spiro).dropna()
all_data = all_data.drop(['Time'],axis=1)
all_data.head()

,OZONE,PM2.5Queens,PM2.5Queens Near-road,AQI,barometricPressure,precipitationLast3Hours,relativeHumidity,temperature,time,PEF(l/min),FEV1(l),unixDate,unixTime
unixDate,,,,,,,,,,,,,
18313.0,30.818182,3.372727,5.772727,22.272727,103010.0,0.0,41.008829,0.0,2020-02-21 00:51:00,415,2.89,18313.0,480.0
18313.0,30.818182,3.372727,5.772727,22.272727,103150.0,0.0,40.326484,-2.2,2020-02-21 03:51:00,388,2.54,18313.0,12660.0
18314.0,22.291667,7.404167,7.479167,27.638889,102200.0,0.0,18.705038,10.0,2020-02-22 19:51:00,476,2.78,18314.0,70320.0
18315.0,19.000000,10.737500,10.116667,34.902778,102300.0,0.0,66.125542,0.6,2020-02-23 05:51:00,294,1.62,18315.0,18780.0
18315.0,19.000000,10.737500,10.116667,34.902778,102340.0,0.0,35.825672,9.4,2020-02-23 15:51:00,384,2.49,18315.0,56220.0


In [14]:
Y = all_data[['PEF(l/min)']]
x_col = list(all_data.columns)
x_col.remove('PEF(l/min)')
x_col.remove('FEV1(l)')
x_col
X = all_data[x_col]
X = sm.add_constant(X)

mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary()
p_values

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [190]:
pollution_breath.head()

,PM2.5,Date,Time,PEF(l/min),FEV1(l),unixDate,unixTime
unixDate,,,,,,,
18314.0,7.479167,2020/2/22,03:31:00,388.0,2.54,18314.0,12660.0
18314.0,7.479167,2020/2/22,00:08:00,415.0,2.89,18314.0,480.0
18315.0,10.116667,2020/2/23,19:32:00,476.0,2.78,18315.0,70320.0
18316.0,13.137500,2020/2/24,05:13:00,294.0,1.62,18316.0,18780.0
18316.0,13.137500,2020/2/24,15:37:00,384.0,2.49,18316.0,56220.0


In [194]:
Y = pollution_breath['PEF(l/min)']
X = pollution_breath['PM2.5']
X = sm.add_constant(X)

mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary()
p_values

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PEF(l/min)   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.5848
Date:                Fri, 27 Mar 2020   Prob (F-statistic):              0.446
Time:                        13:19:02   Log-Likelihood:                -981.12
No. Observations:                 169   AIC:                             1966.
Df Residuals:                     167   BIC:                             1973.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        398.4753     18.503     21.535      0.000     361.944     435.006
PM2.5         -2.0825      2.723     -0.765      0.446      -7.459       3.294
==============================================================================
Omnibus:                        3.957   Durbin-Watson:                   1.804
Prob(Omnibus):                  0.138   Jarque-Bera (JB):                2.611
Skew:                           0.110   Prob(JB):                        0.271
Kurtosis:                       2.432   Cond. No.                         20.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
#multi-linear regression - investigate further
ols = linear_model.LinearRegression()
X = temp_spiro[['unixDate', 'FEV1(l)', 'unixTime','PRCP','TAVG','WT08']]  # input varialbe
Y = temp_spiro['PEF(l/min)']  # output variable
ols.fit(X, Y)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X, Y)))
ols.get_params()

Intercept: -14061.406504
Coefficients: [ 7.68021310e-01  1.46647246e+02 -1.39668447e-04 -2.00085216e+01
  2.94322816e-01 -3.86657168e+00]
R^2: 0.828610


{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [32]:
mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary2()
p_values
#no strong correlation other than FEV

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Ordinary least squares
===============================================================================
Model:                  OLS              Adj. R-squared (uncentered): 0.992    
Dependent Variable:     PEF(l/min)       AIC:                         1322.4903
Date:                   2020-03-25 17:11 BIC:                         1339.8324
No. Observations:       133              Log-Likelihood:              -655.25  
Df Model:               6                F-statistic:                 2853.    
Df Residuals:           127              Prob (F-statistic):          7.65e-133
R-squared (uncentered): 0.993            Scale:                       1166.3   
-----------------------------------------------------------------------------------
              Coef.       Std.Err.        t        P>|t|       [0.025       0.975] 
-----------------------------------------------------------------------------------
unixDate       0.0004       0.0015      0.2523     0.8012      -0.0026       0.0034
FEV1(l)      144.8441       6.1181     23.6747     0.0000     132.7375     156.9507
unixTime      -0.0001       0.0001     -0.9917     0.3232      -0.0003       0.0001
PRCP         -13.2507      13.5745     -0.9761     0.3308     -40.1121      13.6108
TAVG           0.5633       0.4806      1.1721     0.2433      -0.3877       1.5142
WT08         -11.1143       9.7108     -1.1445     0.2546     -30.3302       8.1016
-------------------------------------------------------------------------------
Omnibus:                   52.561           Durbin-Watson:              2.057  
Prob(Omnibus):             0.000            Jarque-Bera (JB):           313.697
Skew:                      1.197            Prob(JB):                   0.000  
Kurtosis:                  10.133           Condition No.:              255361 
===============================================================================
* The condition number is large (3e+05). This might indicate             strong
multicollinearity or other numerical problems.
"""

In [33]:
#same analysis - different weather source
X = newtemp_spiro[['FEV1(l)','DailyAverageDryBulbTemperature','DailyAverageStationPressure','DailyPrecipitation','unixDate','unixTime']]
Y = newtemp_spiro['PEF(l/min)']  # output variable
ols.fit(X, Y)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X, Y)))
ols.get_params()
#very similar to above

Intercept: -16633.004984
Coefficients: [ 1.47431994e+02  1.85513748e-01 -1.69166911e+00 -2.31983848e+01
  9.11260269e-01 -1.39833625e-04]
R^2: 0.828448


{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}